<a href="https://colab.research.google.com/github/LuisPeMoraRod/AI-Laboratories/blob/main/Lab5_MatrixProgrammed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# import libraries
import torch


### Ejercicio 1
Implemente la función *calcularTrazaMatriz*, la cual calcula la traza de una matriz usando únicamente operaciones básicas en Pytorch (multiplicación, multiplicación por elemento, matriz identidad, etc), prescindiendo de estructuras de repetición como el for el while. (10 pts) 


### Ejercicio 2
Desarrolle la tranpuesta de una matriz. (10 pts)

In [ ]:
import numpy as np


# Método que permite obtener la matriz transpuesta.


def transpose_matrix(matrix):

    # Obtener el numero de filas y columnas de la matriz
    num_rows = len(matrix)
    num_cols = len(matrix[0])

    # Creamos una matriz de ceros invertida
    transpose = [[0 for j in range(num_rows)] for i in range(num_cols)]

    # iteramos cada elemento de la matriz junto con la creada
    for i in range(num_rows):
        for j in range(num_cols):
            # Agregamos los datos correspondientes a la matriz tranpuesta
            transpose[j][i] = matrix[i][j]

    return transpose

# Método que permite imprimir las matrices.

def printMatrix(matrix):
  for row in matrix:
    print(row)


matriz = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

print("------------ Método programado ----------------")
print("Matriz Original")
printMatrix(matriz)
transpose = transpose_matrix(matriz)
print("Matriz Transpuesta")
printMatrix(transpose)

print("------------ Con librería ----------------")

print("Matriz Transpuesta")

npmatriz = np.array(matriz).transpose()
printMatrix(npmatriz)

------------ Método programado ----------------
Matriz Original
[1, 2, 3]
[4, 5, 6]
[7, 8, 9]
Matriz Transpuesta
[1, 4, 7]
[2, 5, 8]
[3, 6, 9]
------------ Con librería ----------------
Matriz Transpuesta
[1 4 7]
[2 5 8]
[3 6 9]


### Ejercicio 3
Desarrolle la matriz adjunta de una matriz (10 pts)

In [70]:
class Adj:
  # auxiliar variables used to loop through the Tensor
  i = 0
  j = 0
  def __init__(self, M: torch.Tensor):
    self.M = M
    self.rows = len(M)
    self.col = len(M[0])
    self.ones = torch.ones(self.rows, self.col)
    self.adj = self.get_adj_matrix()

  def __str__(self):
    return str(self.adj)

  def get_minor(self, i : int, j : int):
      """
      Return the minor matrix in relation to indexes i and j
      """
      M = torch.cat((self.M[:i],self.M[i+1:])) # delete row i
      M = torch.transpose(M, 0, 1)
      M = torch.cat((M[:j], M[j+1:])) # delete column j
      M = torch.transpose(M, 0, 1)
      return M

  def get_cofactor(self, element) -> int:
    """
    Returns the cofactor for a specific element in the matrix
    """
    minor = self.get_minor(self.i, self.j)
    determinant = torch.det(minor)
    result = (-element)**(self.i+self.j)*determinant #computes cofactor

    #update indexes
    if (self.j + 1 == len(M[0])):
      self.i+=1
      self.j = 0
    else:
      self.j+=1

    return result

  def get_cofactors_matrix(self) -> torch.Tensor:
    return self.ones.apply_(self.get_cofactor) #applies get_factor method to every element in ones matrix

  def get_adj_matrix(self) -> torch.Tensor:
    cofactor_matrix = self.get_cofactors_matrix()
    return torch.transpose(cofactor_matrix, 0, 1)

M = [[7, 5, 3, 4], [2, 3, 6, 4], [6, 10, 9, 4], [1, 2, 3, 4]]
M = torch.tensor(M, dtype=torch.float64)
print(f'Original matrix:\n{M}')
adj_matrix = Adj(M)
print(f'\n\nAdjugate matrix:\n{adj_matrix}')


Original matrix:
tensor([[ 7.,  5.,  3.,  4.],
        [ 2.,  3.,  6.,  4.],
        [ 6., 10.,  9.,  4.],
        [ 1.,  2.,  3.,  4.]], dtype=torch.float64)


Adjugate matrix:
tensor([[ -72.,  -72.,   36.,  108.],
        [  36.,  144.,  -72., -108.],
        [  12., -132.,   12.,  108.],
        [  -9.,   45.,   18., -135.]])


### Ejercicio 4
Desarrolle la inversa de una matriz. Debe validar si es invertible o no. (20 pts)

In [ ]:
import numpy as np

def inverse_matrix(matrix):
 return [[2, -2, 2], [2, 1, 0], [3, -2, 2]]

matriz = [[2, -2, 2], [2, 1, 0], [3, -2, 2]]

print("------------ Método programado ----------------")
print("Matriz Original")
printMatrix(matriz)
inverse = inverse_matrix(matriz)
print("Matriz Inversa")
printMatrix(inverse)

print("------------ Con librería ----------------")

print("Matriz Inversa")

npmatriz = np.array(matriz)

npmatriz_inv = np.linalg.inv(npmatriz)

printMatrix(npmatriz_inv)

------------ Método programado ----------------
Matriz Original
[2, -2, 2]
[2, 1, 0]
[3, -2, 2]
Matriz Inversa
[2, -2, 2]
[2, 1, 0]
[3, -2, 2]
------------ Con librería ----------------
Matriz Inversa
[-1.00000000e+00  1.48029737e-16  1.00000000e+00]
[ 2.  1. -2.]
[ 3.5  1.  -3. ]


### Ejercicio 5
La Matriz de covarianza. (50 pts) 